In [1]:
import pandas as pd
import urllib.parse 
import numpy as np 
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import psycopg2
import psycopg2.extras as extras


In [2]:
load_dotenv()
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

In [3]:
# import data
#accounts_receivable
acc_rec = pd.read_excel('transactions_upload.xlsx', sheet_name='Accounts receivable ledger')
for col in list(acc_rec.columns):
    if "date" in col:
        acc_rec[col] = pd.to_datetime(acc_rec[col], format='mixed') 
acc_rec = acc_rec.replace({np.nan: None})
acc_rec.head()



,invoice_number,date,customer_name,customer_number,amount,due_date,payment,payment_date,payment_id
0,1,2025-05-01,Planet Express,12038,50000,2025-05-02,None,None,None
1,2,2025-09-01,Mom's Friendly Robot Factory,12000,100000,2025-09-02,None,None,None
2,3,2025-10-01,Romanticorp,12990,73640,2025-10-02,None,None,None
3,4,2025-01-18,Hal Insitute for Criminally Insane Robots,12010,12500,2025-02-18,None,None,None
4,5,2025-01-21,Cookieville Minimum-Security Orphanarium,11900,10000,2025-02-21,None,None,None


In [10]:
# Customers
customers = pd.read_excel('transactions_upload.xlsx', sheet_name='Customers')
for col in list(customers.columns):
    if "date" in col:
        customers[col] = pd.to_datetime(customers[col], format='mixed')
customers = customers.replace({np.nan: None})
customers.head()

,customer_number,customer_name,start_date,end_date,terms,discount_applicable,discount_amount
0,11900,Cookieville Minimum-Security Orphanarium,2024-01-03,None,30,False,0
1,12000,Mom's Friendly Robot Factory,2023-03-25,None,30,False,0
2,12001,Democratic Order of Planets,2024-12-12,None,30,False,0
3,12010,Hal Insitute for Criminally Insane Robots,2024-06-25,None,30,False,0
4,12038,Planet Express,2022-08-18,None,10,False,0


In [5]:
# Payments
payments = pd.read_excel('transactions_upload.xlsx', sheet_name='Payments')
for col in list(payments.columns):
    if "date" in col:
        payments[col] = pd.to_datetime(payments[col], format='mixed') 
payments = payments.replace({np.nan: None})
payments.head()

,transaction_id,payment_date,payment_amount,payment_reference
0,948347,2025-01-31,25000,12038 - 1 - We'll pay the rest later
1,34847,2025-01-20,100000,Customer Nr 12000 - Invoice Nr 2 - Mom's
2,29304,2025-01-02,12500,12010 - 4
3,3837459,2025-10-02,10000,11900 - 5
4,390576,2025-02-14,76000,12990 - Invoice Nr. 3


In [6]:
# Creating connection to DB
engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOSTNAME}:{PORT}/{DB_NAME}')
#conn = psycopg2.connect("host=localhost dbname=agent_test_db user=postgres")
#cur = conn.cursor()

In [7]:
def execute_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()


In [8]:
conn = psycopg2.connect(
    database=DB_NAME, user=USERNAME, password=PASSWORD, host=HOSTNAME, port=PORT
)

In [13]:
execute_values(conn, payments, 'payments')

the dataframe is inserted
